In [2]:
%matplotlib Inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate # For evaluate..
from ast import literal_eval

import warnings 
warnings.simplefilter('ignore')

Simple Recommender


In [4]:
movie_md = pd.read_csv("movies_metadata.csv")
movie_md.head(10)

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
5  False                                                NaN  60000000   
6  False                                                NaN  58000000   
7  False                                                NaN         0   
8  False                                                NaN  35000000   
9  False  {'id': 645, 'name': 'James Bond Collection', '...  58000000   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   
5  [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...   
6  [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...   
7  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
8  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
9  [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...   

                                       homepage     id    imdb_id  \
0          http://toystory.disney.com/toy-story    862  tt0114709   
1                                           NaN   8844  tt0113497   
2                                           NaN  15602  tt0113228   
3                                           NaN  31357  tt0114885   
4                                           NaN  11862  tt0113041   
5                                           NaN    949  tt0113277   
6                                           NaN  11860  tt0114319   
7                                           NaN  45325  tt0112302   
8                                           NaN   9091  tt0114576   
9  http://www.mgm.com/view/movie/757/Goldeneye/    710  tt0113189   

  original_language               original_title  \
0                en                    Toy Story   
1                en                      Jumanji   
2                en             Grumpier Old Men   
3                en            Waiting to Exhale   
4                en  Father of the Bride Part II   
5                en                         Heat   
6                en                      Sabrina   
7                en                 Tom and Huck   
8                en                 Sudden Death   
9                en                    GoldenEye   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   
5  Obsessive master thief, Neil McCauley leads a ...  ...   1995-12-15   
6  An ugly duckling having undergone a remarkable...  ...   1995-12-15   
7  A mischievous young boy, Tom Sawyer, witnesses...  ...   1995-12-22   
8  International action superstar Jean Claude Van...  ...   1995-12-22   
9  James Bond must unmask the mysterious head of ...  ...   1995-11-16   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': '

In [4]:
movie_md['genres'] = movie_md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else[]) # type: ignore

In [5]:
vote_counts = movie_md[movie_md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movie_md[movie_md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [6]:
m = vote_counts.quantile(0.95)
m

434.0

In [8]:
movie_md['year']=pd.to_datetime(movie_md['release_date'],errors='coerce').apply(lambda  x: str(x).split('-')[0] if x!= np.nan else np.nan)

In [9]:
qualified = movie_md[(movie_md['vote_count'] >= m) & (movie_md['vote_count'].notnull()) & (movie_md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

Setup the weighted Rating Formula 
 (v/(v+m).R)+(m/(v+m).C)

In [18]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return(v/(v+m)*R + (m/(m+v)*C))

In [20]:
qualified['wr'] = qualified.apply(weighted_rating,axis=1,result_type='expand')
qualified = qualified.sort_values('wr',ascending=False).head(250)

See the results


In [21]:
qualified.head(10)

title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   
292                                         Pulp Fiction  1994        8670   
314                             The Shawshank Redemption  1994        8358   
7000       The Lord of the Rings: The Return of the King  2003        8226   
351                                         Forrest Gump  1994        8147   
5814               The Lord of the Rings: The Two Towers  2002        7641   

       vote_average  popularity  \
15480             8   29.108149   
12481             8  123.167259   
22879             8   32.213481   
2843              8   63.869599   
4863              8   32.070725   
292               8  140.950236   
314               8   51.645403   
7000              8   29.324358   
351               8   48.307194   
5814              8   29.423537   

                                                  genres        wr  
15480  [Action, Thriller, Science Fiction, Mystery, A...  7.917588  
12481                   [Drama, Action, Crime, Thriller]  7.905871  
22879                [Adventure, Drama, Science Fiction]  7.897107  
2843                                             [Drama]  7.881753  
4863                        [Adventure, Fantasy, Action]  7.871787  
292                                    [Thriller, Crime]  7.868660  
314                                       [Drama, Crime]  7.864000  
7000                        [Adventure, Fantasy, Action]  7.861927  
351                             [Comedy, Drama, Romance]  7.860656  
5814                        [Adventure, Fantasy, Action]  7.851924

In [22]:
s = movie_md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre' # Not according to title but genre as level= 1
gen_movie_md = movie_md.drop('genres',axis = 1).join(s)

In [23]:
# New chart
def build_chart(genre,percentile = 0.85):
    df = gen_movie_md[gen_movie_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified 


In [24]:
build_chart('Drama').head(14)

title  year  vote_count  vote_average  \
10309  Dilwale Dulhania Le Jayenge  1995         661             9   
12481              The Dark Knight  2008       12269             8   
22879                 Interstellar  2014       11187             8   
2843                    Fight Club  1999        9678             8   
314       The Shawshank Redemption  1994        8358             8   
351                   Forrest Gump  1994        8147             8   
834                  The Godfather  1972        6024             8   
24860           The Imitation Game  2014        5895             8   
359                  The Lion King  1994        5520             8   
18465             The Intouchables  2011        5410             8   
22841     The Grand Budapest Hotel  2014        4644             8   
586       The Silence of the Lambs  1991        4549             8   
11354                 The Prestige  2006        4510             8   
522               Schindler's List  1993        4436             8   

       popularity        wr  
10309   34.457024  8.607205  
12481  123.167259  7.983111  
22879   32.213481  7.981489  
2843    63.869599  7.978628  
314     51.645403  7.975286  
351     48.307194  7.974653  
834     41.109264  7.965843  
24860    31.59594  7.965106  
359     21.605761  7.962771  
18465   16.086919  7.962025  
22841   14.442048  7.955873  
586      4.307222  7.954968  
11354    16.94556  7.954586  
522     41.725123  7.953842

Content based Recommender

To personalise our recommendations more, I am going to build an engine that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. Since we will be using movie metadata (or content) to build this engine, this also known as **Content Based Filtering**
I will build two Content Based Recommenders based on:

    Movie Overviews and Taglines
    Movie Cast, Crew, Keywords and Genre


In [3]:
Link_small = pd.read_csv("links_small.csv")
Link_small = Link_small[Link_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [5]:
movie_md = movie_md.drop([19730, 29503, 35587])

In [6]:
movie_md['id'] = movie_md['id'].astype('int')

In [7]:
smd = movie_md[movie_md['id'].isin(Link_small)]
smd.shape

(9099, 24)

Movie description based recommender

In [8]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [9]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [10]:
tfidf_matrix.shape

(9099, 268124)

Cosine Similarity

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

cosine(x,y)=x.y⊺//||x||.||y||

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster

In [11]:
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [12]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index,index = smd['title'])

In [13]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [23]:
get_recommendations('Forrest Gump').head(10)

8435                           Frozen
3764                      Serendipity
353                     Jason's Lyric
655                        Phenomenon
7980                    Perfect Sense
7856                      Project Nim
4425                      City of God
3318    Long Night's Journey Into Day
5040                        Jane Eyre
5439                     The Notebook
Name: title, dtype: object

metadata Based Recommender

To build our standard metadata based content recommender, we will need to merge our current dataset with the crew and the keyword datasets. Let us prepare this data as our first step.

In [18]:
credit = pd.read_csv('credits.csv')
keyword = pd.read_csv('keywords.csv')
keyword['id'] = keyword['id'].astype('int')
credit['id'] = credit['id'].astype('int')
movie_md['id'] = movie_md['id'].astype('int')

In [19]:
movie_md.shape

(45463, 24)

In [21]:
movie_md = movie_md.merge(credit, on='id')
movie_md = movie_md.merge(keyword, on='id')
smd = movie_md[movie_md['id'].isin(Link_small)]
smd.shape

(9663, 30)

We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

- **Crew**: From the crew, we will only pick the director as our feature since the others don't contribute that much to the feel of the movie
- **Cast**:Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list.

In [32]:
#smd.drop(['cast_x','crew_x'],axis=1,inplace=True)
smd = smd.rename(columns={'crew_y':'crew','cast_y':'cast'},inplace=True)

In [33]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keyword'] = smd['keyword'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

TypeError: 'NoneType' object is not subscriptable